In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [4]:
# use gpu for trainning when available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # convert a 2D 28*28 image into a contiguous array of 784 pixel values
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            # The linear layer is a module that applies a linear transformation on the input using it’s stored weights and biases.
            nn.Linear(28*28, 512),
            # Non-linear activations are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.
            nn.ReLU(),
            nn.Linear(512, 512),
            # ReLU sets all the negative values to 0 and keep the positive values
            nn.ReLU(),
            # returns logits - raw values in [-infty, infty]
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [5]:
# Given an input, we can get a 10-dim tensor which stands for probability distribution
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits)
# get the prediction probabilities by passing it through an instance of the nn.Softmax module.
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[0.0622, 0.0000, 0.0376, 0.0670, 0.0182, 0.0000, 0.0324, 0.0000, 0.0000,
         0.0120]], device='cuda:0', grad_fn=<ReluBackward0>)
Predicted class: tensor([3], device='cuda:0')


In [6]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0225, -0.0095,  0.0205,  ..., -0.0246,  0.0275, -0.0315],
        [-0.0089, -0.0357, -0.0090,  ...,  0.0068,  0.0106, -0.0327]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0214,  0.0027], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-2.3879e-02, -3.1113e-02, -7.0927e-03,  ..., -9.8275e-03,
          9.9797e-05,  4.0105e-02],
        [-2.3977e-02,  3.8766e-02, -1.1695e-02,  ...,  3.6612e-03,
          2.0632e-02,  2.4773e-04]], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([0.0062, 0.0051], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0105, -0.0159, -0.0224,  ..., -0.0403, -

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [33]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300164  [    0/60000]
loss: 2.296530  [ 6400/60000]
loss: 2.283843  [12800/60000]
loss: 2.282676  [19200/60000]
loss: 2.275575  [25600/60000]
loss: 2.247028  [32000/60000]
loss: 2.261315  [38400/60000]
loss: 2.241509  [44800/60000]
loss: 2.221464  [51200/60000]
loss: 2.224913  [57600/60000]
Test Error: 
 Accuracy: 35.3%, Avg loss: 0.034907 

Epoch 2
-------------------------------
loss: 2.217163  [    0/60000]
loss: 2.229348  [ 6400/60000]
loss: 2.208843  [12800/60000]
loss: 2.216443  [19200/60000]
loss: 2.206436  [25600/60000]
loss: 2.146705  [32000/60000]
loss: 2.180224  [38400/60000]
loss: 2.145914  [44800/60000]
loss: 2.103900  [51200/60000]
loss: 2.119587  [57600/60000]
Test Error: 
 Accuracy: 43.5%, Avg loss: 0.033303 

Epoch 3
-------------------------------
loss: 2.107449  [    0/60000]
loss: 2.128494  [ 6400/60000]
loss: 2.096261  [12800/60000]
loss: 2.117387  [19200/60000]
loss: 2.107770  [25600/60000]
loss: 2.012576  [32000/600

In [34]:
# PyTorch also has native ONNX export support. There are a lot of things you can do with ONNX model, including running inference on different platforms and in different programming languages
torch.save(model.state_dict(), "../models/FashionMNIST_20Ep.model")
print("Saved PyTorch Model State to FashionMNIST_20Ep.model")

Saved PyTorch Model State to FashionMNIST_20Ep.model


In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load("../models/FashionMNIST_20Ep.model"))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[2][0], test_data[2][1]
for i in range(100):
    x, y = test_data[i][0], test_data[i][1]
    with torch.no_grad():
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Sneaker", Actual: "Ankle boot"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Dress", Actual: "Trouser"
Predicted: "Dress", Actual: "Trouser"
Predicted: "Pullover", Actual: "Shirt"
Predicted: "Dress", Actual: "Trouser"
Predicted: "Coat", Actual: "Coat"
Predicted: "Coat", Actual: "Shirt"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Pullover", Actual: "Coat"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Dress", Actual: "Dress"
Predicted: "Coat", Actual: "Coat"
Predicted: "Dress", Actual: "Trouser"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Pullover", Actual: "Coat"
Predicted: "Bag", Actual: "Bag"
Predicted: "T-shirt/top", Actual: "T-shirt/top"
Predicted: "T-shirt/top", Actual: "Pullover"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Sneaker", Actual: "Ankle boot"
Predicted: "Dress", Actual: "Trouser"
Predicted: "Pullover", A